# Setting up Environment

In [ ]:
import os
import sys
# Set Path
path_wd = '/content/drive/MyDrive/Github/Content'
os.chdir(path_wd)

!pip install -r sources/ChinaFilm/requirements.txt
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [2]:
!#Load Libraries
import gc
import pandas as pd
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize Registration Class
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
os.chdir('sources/ChinaFilm')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [9]:
from Record_Registration import Registration
dy_reg = Registration(driver)
gc.collect()

df = dy_reg.Refined_Records(update_first=False)
curr_issue_name = df.sort_values(['公示日期', '公示批次名称'], ascending=False)['公示批次名称'].iloc[0]
df_curr =  df.loc[df.公示批次名称 == curr_issue_name,]
del df
gc.collect()
df_curr.columns

Index(['备案立项号', '片名', '备案单位', '编剧', '备案结果', '备案地', '梗概', '公示日期', '公示批次名称',
       '备案详细页链接', '公示批次链接', '公示年', '公示批次起始', '类型', '备案申请年份', '备案立项年度顺序号',
       '公示覆盖期间', '公示覆盖天数'],
      dtype='object')

# XXXX年XX月X日，XXXX年XX月XXXX处理完毕的备案公示新鲜出炉！ 这一批次中，

In [ ]:
df_curr['公示日期'] = df_curr['公示日期'].astype('datetime64')
name_earliest = df_curr.sort_values(['备案申请年份', '备案立项年度顺序号']).iloc[0]['片名']
regid_earliest = df_curr.sort_values(['备案申请年份', '备案立项年度顺序号']).iloc[0]['备案立项号']
name_latest = df_curr.sort_values(['备案申请年份', '备案立项年度顺序号']).iloc[df_curr.shape[0]-1]['片名']
regid_latest = df_curr.sort_values(['备案申请年份', '备案立项年度顺序号']).iloc[df_curr.shape[0]-1]['备案立项号']

txt = str(df_curr[:1]['公示日期'].dt.date.iloc[0].year) + '年' 
txt += str(df_curr[:1]['公示日期'].dt.date.iloc[0].month) + '月'
txt += str(df_curr[:1]['公示日期'].dt.date.iloc[0].day) + '日，'
txt += df_curr.loc[0,'公示批次起始'][0] + '年' + df_curr.loc[0,'公示批次起始'][1] + '月'
txt += df_curr.loc[0,'公示批次起始'][2]
txt += '处理完毕的备案公示新鲜出炉！共计{}部影片。'.format(df_curr.shape[0])
txt += '这一批次中，最遥远的项目是《{}》,'.format(name_earliest)
txt += '备案号为{},'.format(regid_earliest)
txt += '最近期的项目是《{}》,'.format(name_latest)
txt += '备案号为{}。'.format(regid_latest)
txt += '\n'
print(txt)

2020年11月3日，2020年05月整月处理完毕的备案公示新鲜出炉！共计286部影片。这一批次中，最遥远的项目是《终极心跳》,备案号为影剧备字[2017]第3452号,最近期的项目是《扣杀》,备案号为影剧备字[2020]第7310号。



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
name_earliest = df_curr.sort_values(['备案申请年份', '备案立项年度顺序号']).iloc[0]['片名']
regid_earliest = df_curr.sort_values(['备案申请年份', '备案立项年度顺序号']).iloc[0]['备案立项号']

#备案中，国家电影局直备的有XX部，各地备案量排名前三的分别是XXX、XXX、XXX。
补备案的共有两部，先拍片、再备案属于高风险行为，切记慎之又慎。

In [ ]:
df_location = df_curr.groupby('备案地')['备案地'].count().sort_values(ascending=False).rename('数量').reset_index()
txt2 = '备案中，国家电影局直备的有{}部,'.format(df_location[df_location.备案地 == '直备'].iloc[0]['数量'])
txt2 += '各地备案量排名前三的分别是{}{}部、{}{}部和{}{}部。'.format(df_location.loc[0]['备案地'], df_location.loc[0]['数量'], 
                                         df_location.loc[1]['备案地'], df_location.loc[2]['数量'],
                                         df_location.loc[1]['备案地'], df_location.loc[2]['数量'])
txt2 +=  '补备案的共有{}部，先拍片、再备案属于高风险行为，切记慎之又慎。'.format(df_location[df_location.备案地.str.contains('(补)')].loc[:,'数量'].sum())


print(txt2)

备案中，国家电影局直备的有15部,各地备案量排名前三的分别是北京市54部、上海市29部和上海市29部。补备案的共有2部，先拍片、再备案属于高风险行为，切记慎之又慎。


/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


#这个周期内，有{}家影视公司至少完成了两部影片的备案，堪称劳模。
本周期最勤奋的影视公司有XXX、XXX及XXX，分别完成了XXX、XXX及XXX个备案。

In [ ]:
pat_story = '|'.join(['故事片','合拍片'])
df_2plus_co = df_curr[df_curr['类型'].str.contains(pat1)].groupby(['备案单位'])['备案单位'].count().sort_values(ascending=False).rename('数量').to_frame()
pat_2plus = '|'.join(df_2plus_co[df_2plus_co.数量 > 1].index.tolist())
df_2plus_ls = df_curr.loc[df_curr['备案单位'].str.contains(pat_2plus) & df_curr['类型'].str.contains(pat_story)][['备案单位','片名',]]
df_2plus_ls['影片'] = df_2plus_ls.groupby('备案单位').transform(lambda x: ','.join('《'+ x + '》'))
df_2plus_ls = df_2plus_ls[['备案单位', '影片']].drop_duplicates()

##### TEXT ##### 
txt3 = '这个周期内，有{}家影视公司至少完成了两部故事片（含合拍片，不含动画片）的备案，堪称劳模, 值得上榜表扬。'.format(df_2plus_co[df_2plus_co.数量 >1].count()[0])

# change to visual table view later
for _, row in df_2plus_ls.iterrows():
  txt3 += '\n{}: {};'.format(row['备案单位'], row['影片'])
print(txt3)

这个周期内，有9家影视公司至少完成了两部故事片（含合拍片，不含动画片）的备案，堪称劳模, 值得上榜表扬。
天马影联影视文化（北京）有限公司、东方影业出品有限公司: 《一路同行》,《拼命三郎》
山西天禄堂影视制作有限公司: 《大背棍》,《香飘四季》
河南独立文化传播有限公司: 《临时乘客》,《为梦而来》
广州市广播电视台: 《范蠡献西施（粤剧）》,《新搜书院（粤剧）》
广东博纳影业传媒有限公司: 《中国·医生》,《钦差大臣》
福建风林火山影视文化传播有限公司: 《我的爸爸是英雄》,《烟雨红影》
北京沃星影业有限公司: 《藏地奇兵》,《鲁门诅咒》
北京海韵晨耕文化传播有限公司: 《御丹传》,《青春梦想时》
北京大卫之星文化传播有限公司: 《大漠往事》,《26年》


,备案单位,影片
16,天马影联影视文化（北京）有限公司、东方影业出品有限公司,"《一路同行》,《拼命三郎》"
103,山西天禄堂影视制作有限公司,"《大背棍》,《香飘四季》"
153,河南独立文化传播有限公司,"《临时乘客》,《为梦而来》"
176,广州市广播电视台,"《范蠡献西施（粤剧）》,《新搜书院（粤剧）》"
183,广东博纳影业传媒有限公司,"《中国·医生》,《钦差大臣》"
215,福建风林火山影视文化传播有限公司,"《我的爸爸是英雄》,《烟雨红影》"
233,北京沃星影业有限公司,"《藏地奇兵》,《鲁门诅咒》"
245,北京海韵晨耕文化传播有限公司,"《御丹传》,《青春梦想时》"
275,北京大卫之星文化传播有限公司,"《大漠往事》,《26年》"


In [ ]:
df_type = df_curr.groupby('类型')['类型'].count().sort_values(ascending=False).rename('数量').to_frame()

,数量
类型,
故事片,223
纪录片,23
动画片,19
科教片,14
合拍片,5
特种片,2


In [ ]:
# 上市公司相关 ##
info_public_film_co = pd.read_csv('/content/drive/MyDrive/Github/Content/tools/articles/reference/info_public_film_co.csv', index_col=0, encoding='utf-8-sig')
ls = []
for row in info_public_film_co['公司简称'].apply(eval).tolist():
  ls.extend(row)
ls = list(set(ls))
pat_public = '|'.join(ls)
pat_public += '|阿里|腾讯|优酷|爱奇艺|博纳'
df_public_co = df_curr.loc[df_curr['备案单位'].str.contains(pat_public)]
df_public_project = df_curr.loc[df_curr['备案单位'].str.contains(pat_public) & df_curr['类型'].str.contains(pat_story)][['片名', '备案单位', '编剧', '梗概']]

txt4 = '在{}个备案中, 预计有{}部影片获取了上市影视公司的投资，'.format(df_curr.shape[0], df_public_co.shape[0])
txt4 += '占本次备案的{}%。'.format(round(df_public_co.shape[0]/df_curr.shape[0]*100, 2))

txt4 += '经过AI机器人福菜菜的电眼识别，选出一些本期备案中较有实力的项目:'
i = 1
for _, row in df_public_project.iterrows():
  txt4 += '\n({})《{}》,编剧：{}\n'.format(i, row['片名'].lstrip(' ').rstrip(' '), row['编剧'])
  txt4 += '公司：{}\n'.format(row['备案单位'])
  txt4 += '梗概: {}\n'.format(row['梗概'])
  i += 1
print(txt4)

在286个备案中, 预计有16部影片获取了上市影视公司的投资，占本次备案的5.59%。经过AI机器人福菜菜的电眼识别，选出一些本期备案中较有实力的项目:
(1)《阿卡贝拉合唱团》,编剧：杨昆
公司：中国电影股份有限公司
梗概: 本片灵感来源于厦门六中合唱团真实事例。音乐学博士曲大卫接手了一个人心涣散的高中合唱团，来参加的学生五花八门。同学们讨厌枯燥的发声训练，却对厦门六中的阿卡贝拉合唱兴趣十足，曲大卫不得不改变他的训练计划…

(2)《功夫茶茶》,编剧：张磊
公司：中国电影股份有限公司、北京果实创想影视文化有限公司
梗概: 影片根据来自汕头大学的四名90后女生组成的“功夫茶茶”船队成功横渡大西洋并打破了赛会女队纪录这一真实事件改编。

(3)《一点就到家》,编剧：张冀、周运海、蒋峰
公司：北京嘉映春天影业有限公司、我们制作有限公司、亚博思文化传媒(北京）有限公司、阿里巴巴影业（北京）有限公司、中国电影股份有限公司 、云南民族电影制片厂、华夏电影发行有限责任公司、抖音文化（厦门）有限公司、浙江横店影业有限公司、上海翎刻闪耀影视制作有限公司、北京联瑞影业有限公司、阿那亚影视文化有限公司、北京嘉映文化传媒有限公司
梗概: 机缘巧合下，魏晋北和彭秀兵踏上了从大城市回到云南千年古寨的征途，并与李绍群结识，三人共同创业。与古寨“格格不入”的他们用真诚改变了所有人，开启了一段非常疯狂、纯真而且荒诞的梦想之旅。

(4)《谁说我不靠谱》,编剧：李彬
公司：山西芸韵百纳影视文化有限公司、东阳文纳影视文化传播有限公司
梗概: 不靠谱的梦羽。回国过起了单身狗的生活。梦羽和闺蜜蒲云深入社会基层调查养老现状时，偶遇义务助养老人的方涵，在蒲云的帮助下，梦羽加入了方涵的团队，做起了助养老人的事业。梦羽展开了对方涵疯狂且又独特的追求。

(5)《荡涤乾坤》,编剧：周文鹏
公司：山东法闻信息科技有限公司、中视元媒（北京）影视文化传媒有限公司、小西天时代（北京）影业有限公司、北京尚驰投资有限公司
梗概: 法官李清扬在审理一起杀人案中,银行经理被杀，巨款消失，一系列阴谋暗潮涌动。经侦破，犯罪集团浮出水面，用制造血案、绑架人质等手段，使法官和警察陷入危机，最终罪犯被绳之以法。

(6)《这个姐姐有点狠》,编剧：韩琳、潘元甲
公司：宁夏中视影业有限公司
梗概: 韩晴的爱人莫名失踪，公司被人出卖，有人逼迫她签

In [ ]:
df_animation = df_curr[df_curr['类型'] == '动画片']

各上市影视公司也都（暗自努力/不甘示弱/辛勤耕耘），完成了不少备案。

In [ ]:
# 本周期最高产的编剧有XXX。

In [ ]:

txt02 = '这一期最活跃的影视公司有{1}，

txt02 = '由上市影视公司备案的有
txt03 = '万达'
txt04 = '光线'
txt05 =

txt03 = '这一电影备案中尤为突出的关键词有。'